In [1]:
import numpy as np
import sympy as sp

In [2]:
def criar_vetor_de_simbolos(tamanho):
    """
    Cria um vetor (lista) de símbolos SymPy com o tamanho especificado.

    Parameters:
    tamanho (int): O número de símbolos a serem criados.

    Returns:
    list: Lista contendo os símbolos criados.
    """
    # Cria os símbolos usando sp.symbols
    return sp.symbols(f'x1:{tamanho+1}')

In [3]:
# Função para calcular o produto Kronecker considerando binário exclusivo
def kronecker_binary(vectors):
    result = np.array([1])  # Inicia com o elemento neutro da multiplicação
    for v in vectors:
        result = np.kron(result, v)
    return result

In [4]:
def decimal_para_binario_vetor(numero_decimal, tamanho_bits=8):
    """
    Converte um número decimal para um vetor binário usando produtos de Kronecker.

    Args:
        numero_decimal (int): Número decimal a ser convertido.
        tamanho_bits (int, opcional): Número de bits para representar o número binário. Padrão é 8.

    Returns:
        np.ndarray: Vetor resultante da conversão.
    
    Raises:
        ValueError: Se o número decimal for negativo ou se o tamanho de bits for insuficiente.
    """
    # Mapeamento dos caracteres binários para os vetores desejados
    mapeamento = {'0': [1, 0], '1': [0, 1]}

    # Verifica se o número é um inteiro não negativo
    if not isinstance(numero_decimal, int):
        print(numero_decimal)
        raise TypeError("O número decimal deve ser um inteiro.")
    if numero_decimal < 0:
        raise ValueError("A função não suporta números decimais negativos.")

    # Converter o número decimal para binário e remover o prefixo '0b'
    numero_binario = bin(numero_decimal)[2:]

    # Verifica se o número binário cabe no tamanho de bits desejado
    if len(numero_binario) > tamanho_bits:
        raise ValueError(f"O número binário '{numero_binario}' excede o tamanho de bits especificado ({tamanho_bits} bits).")

    # Adiciona zeros à esquerda para completar o tamanho de bits desejado
    numero_binario = numero_binario.zfill(tamanho_bits)

    # Inicializa o vetor com 1 para o produto de Kronecker
    vetor = np.array([1])
    
    # Aplica o produto de Kronecker para cada dígito binário
    for digito in numero_binario:
        vetor = np.kron(vetor, mapeamento[digito])

    return vetor

In [5]:
bits = 4
variaveis = sp.Matrix(kronecker_binary(np.array_split(criar_vetor_de_simbolos(bits*2), bits)))
variaveis

Matrix([
[x1*x3*x5*x7],
[x1*x3*x5*x8],
[x1*x3*x6*x7],
[x1*x3*x6*x8],
[x1*x4*x5*x7],
[x1*x4*x5*x8],
[x1*x4*x6*x7],
[x1*x4*x6*x8],
[x2*x3*x5*x7],
[x2*x3*x5*x8],
[x2*x3*x6*x7],
[x2*x3*x6*x8],
[x2*x4*x5*x7],
[x2*x4*x5*x8],
[x2*x4*x6*x7],
[x2*x4*x6*x8]])

In [6]:
import pandas as pd

data = {
  "S2": ["0", "0", "0", "0", "0", "0", "0", "1", "0", "0", "1", "1", "0", "1", "1", "1"],
  "S1": ["0", "0", "1", "1", "0", "1", "1", "0", "1", "1", "0", "0", "1", "0", "0", "1"],
  "S0": ["0", "1", "0", "1", "1", "0", "1", "0", "0", "1", "0", "1", "1", "0", "1", "0"]
}

tabela_verdade = pd.DataFrame(data)
tabela_verdade = tabela_verdade.astype(int)
#tabela_verdade = tabela_verdade[tabela_verdade.columns[::-1]]
tabela_verdade

,S2,S1,S0
0,0,0,0
1,0,0,1
2,0,1,0
3,0,1,1
4,0,0,1
5,0,1,0
6,0,1,1
7,1,0,0
8,0,1,0
9,0,1,1


In [7]:
print(tabela_verdade.dtypes)

S2    int64
S1    int64
S0    int64
dtype: object


In [8]:
vetor_dec = np.array([int("".join(map(str, row)), 2) for row in tabela_verdade.to_numpy()])
vetor_dec

array([0, 1, 2, 3, 1, 2, 3, 4, 2, 3, 4, 5, 3, 4, 5, 6])

In [9]:
vetores_binarios = []
tamanho_bits = tabela_verdade.columns.size

for numero in vetor_dec:
    try:
        vetor = decimal_para_binario_vetor(int(numero), tamanho_bits)
        vetores_binarios.append(vetor)
    except (TypeError, ValueError) as e:
        print(f"Erro ao converter o número {numero}: {e}")

vetores_binarios = sp.Matrix(np.array(vetores_binarios).T)
vetores_binarios

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [10]:
vetores_binarios.shape

(8, 16)

In [11]:
import latexify
from IPython.display import display, Latex

A = sp.Matrix(vetores_binarios)
identity_matrix = sp.eye(A.cols)
identity_matrix

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [12]:
@latexify.function(reduce_assignments=True, use_math_symbols=True)
def matrix_column_product(matrix, column_vector):
    """
    Retorna o produto de 'matrix' por 'column_vector'.
    """
    return matrix * column_vector

# Loop pelas colunas da identidade
for i in range(A.cols):
    print(i)
    result_expr = matrix_column_product(A, identity_matrix.col(i))
    pos_1 = result_expr.tolist().index([1])
    # Monta e exibe a equação completa em LaTeX
    display(Latex(
        rf"$$ {sp.latex(A)} \;\cdot\; {sp.latex(identity_matrix.col(i))}"
        rf" = {sp.latex(result_expr)} = {sp.latex(pos_1)} $$"
    ))

0


<IPython.core.display.Latex object>

1


<IPython.core.display.Latex object>

2


...

<IPython.core.display.Latex object>

14


<IPython.core.display.Latex object>

15


<IPython.core.display.Latex object>

In [13]:
regra = sp.Matrix(np.dot(vetores_binarios, variaveis)) # expressões simbólicas
regra

Matrix([
[                                          x1*x3*x5*x7],
[                            x1*x3*x5*x8 + x1*x4*x5*x7],
[              x1*x3*x6*x7 + x1*x4*x5*x8 + x2*x3*x5*x7],
[x1*x3*x6*x8 + x1*x4*x6*x7 + x2*x3*x5*x8 + x2*x4*x5*x7],
[              x1*x4*x6*x8 + x2*x3*x6*x7 + x2*x4*x5*x8],
[                            x2*x3*x6*x8 + x2*x4*x6*x7],
[                                          x2*x4*x6*x8],
[                                                    0]])

In [28]:
import sympy as sp

def criar_substituicoes(bits):
    simb = criar_vetor_de_simbolos(bits * 2)
    bools = sp.symbols(f'b1:{bits+1}')    # (b1, b2, b3, b4)
    subs = {}
    for i in range(bits):
        zero_sym, one_sym = simb[2*i], simb[2*i+1]
        b = bools[i]
        subs[ zero_sym ] = 1 - b   # coordenada “0” → 1-b
        subs[ one_sym  ] = b       # coordenada “1” →      b
    return subs


In [37]:
print(bits)
subs = criar_substituicoes(bits)
subs

4


{x1: 1 - b1,
 x2: b1,
 x3: 1 - b2,
 x4: b2,
 x5: 1 - b3,
 x6: b3,
 x7: 1 - b4,
 x8: b4}

In [40]:
f_booleana = sp.simplify(regra.subs(subs))
f_booleana

Matrix([
[                                                                  (b1 - 1)*(b2 - 1)*(b3 - 1)*(b4 - 1)],
[                                                       (b1 - 1)*(b3 - 1)*(-b2*(b4 - 1) - b4*(b2 - 1))],
[             -b1*(b2 - 1)*(b3 - 1)*(b4 - 1) + b2*b4*(b1 - 1)*(b3 - 1) - b3*(b1 - 1)*(b2 - 1)*(b4 - 1)],
[b1*b2*(b3 - 1)*(b4 - 1) + b1*b4*(b2 - 1)*(b3 - 1) + b2*b3*(b1 - 1)*(b4 - 1) + b3*b4*(b1 - 1)*(b2 - 1)],
[                                     -b1*b2*b4*(b3 - 1) + b1*b3*(b2 - 1)*(b4 - 1) - b2*b3*b4*(b1 - 1)],
[                                                                           b1*b3*(-2*b2*b4 + b2 + b4)],
[                                                                                          b1*b2*b3*b4],
[                                                                                                    0]])

In [39]:
from itertools import product

# Gerar todas as combinações de 0 e 1 para as 4 variáveis
entradas = list(product([0, 1], repeat=4))
entradas

[(0, 0, 0, 0),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 1, 1),
 (0, 1, 0, 0),
 (0, 1, 0, 1),
 (0, 1, 1, 0),
 (0, 1, 1, 1),
 (1, 0, 0, 0),
 (1, 0, 0, 1),
 (1, 0, 1, 0),
 (1, 0, 1, 1),
 (1, 1, 0, 0),
 (1, 1, 0, 1),
 (1, 1, 1, 0),
 (1, 1, 1, 1)]

In [41]:
for i, e in enumerate(f_booleana):
    # Ordena os símbolos por nome para manter consistência
    variaveis_ordenadas = sorted(e.free_symbols, key=lambda s: s.name)
    
    # Exibe a forma simbólica da função
    nome_funcao = f"F_{{{i+1}}}({', '.join(str(s) for s in variaveis_ordenadas)})"
    display(Latex(rf"$$ {nome_funcao} = {sp.latex(e)} $$"))
    
    bin_valor = []

    for linha_vals in entradas:
        # Mapeia os valores de entrada para os símbolos
        subs_dict = {var: linha_vals[j] for j, var in enumerate(variaveis_ordenadas)}
        entrada_str = ",".join(str(v) for v in linha_vals)
        
        # Constrói a expressão em LaTeX com substituição literal
        expr_latex = sp.latex(e)
        for var in variaveis_ordenadas:
            expr_latex = expr_latex.replace(sp.latex(var), f"({subs_dict[var]})")
        
        # Calcula o valor real da expressão
        resultado = e.subs(subs_dict)
        bin_valor.append(resultado)
        
        # Exibe a linha: substituição literal e resultado
        display(Latex(
            rf"$$ F_{{{i+1}}}({entrada_str}) = {expr_latex} = {sp.latex(resultado)} $$"
        ))

    # Inverte os valores da função para comparar com a tabela verdade
    bin_valor = bin_valor[::-1]
    
    # Verificando se teste é igual a alguma coluna
    for coluna in tabela_verdade.columns:
        if tabela_verdade[coluna].tolist() == bin_valor:
            display(Latex(rf"$$ \text{{Coluna }} {i} \text{{ é igual a }} {sp.latex(e)} $$"))



<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [43]:
for i, e in enumerate(f_booleana):
    variaveis_ordenadas = sorted(e.free_symbols, key=lambda s: s.name)
    print(f"F{i+1}({", ".join(str(s) for s in variaveis_ordenadas)}) = {e}")
    print("-------------------------")
    bin_valor = []
    for vals in entradas:
        valor = e.subs({var: vals[i] for i, var in enumerate(variaveis_ordenadas)})
        bin_valor.append(valor)
        print(f"{vals[0]} {vals[1]} {vals[2]} {vals[3]} | {valor}")
    
    # Verificando se teste é igual a alguma coluna
    for coluna in tabela_verdade.columns:
        if tabela_verdade[coluna].tolist() == bin_valor:
            print(f"Coluna '{coluna}' é igual a",e)
    else:
        print("Nenhuma coluna é igual a",e)

F1(b1, b2, b3, b4) = (b1 - 1)*(b2 - 1)*(b3 - 1)*(b4 - 1)
-------------------------
0 0 0 0 | 1
0 0 0 1 | 0
0 0 1 0 | 0
0 0 1 1 | 0
0 1 0 0 | 0
0 1 0 1 | 0
0 1 1 0 | 0
0 1 1 1 | 0
1 0 0 0 | 0
1 0 0 1 | 0
1 0 1 0 | 0
1 0 1 1 | 0
1 1 0 0 | 0
1 1 0 1 | 0
1 1 1 0 | 0
1 1 1 1 | 0
Nenhuma coluna é igual a (b1 - 1)*(b2 - 1)*(b3 - 1)*(b4 - 1)
F2(b1, b2, b3, b4) = (b1 - 1)*(b3 - 1)*(-b2*(b4 - 1) - b4*(b2 - 1))
-------------------------
0 0 0 0 | 0
0 0 0 1 | 1
0 0 1 0 | 0
0 0 1 1 | 0
0 1 0 0 | 1
0 1 0 1 | 0
0 1 1 0 | 0
0 1 1 1 | 0
1 0 0 0 | 0
1 0 0 1 | 0
1 0 1 0 | 0
1 0 1 1 | 0
1 1 0 0 | 0
1 1 0 1 | 0
1 1 1 0 | 0
1 1 1 1 | 0
Nenhuma coluna é igual a (b1 - 1)*(b3 - 1)*(-b2*(b4 - 1) - b4*(b2 - 1))
F3(b1, b2, b3, b4) = -b1*(b2 - 1)*(b3 - 1)*(b4 - 1) + b2*b4*(b1 - 1)*(b3 - 1) - b3*(b1 - 1)*(b2 - 1)*(b4 - 1)
-------------------------
0 0 0 0 | 0
0 0 0 1 | 0
0 0 1 0 | 1
0 0 1 1 | 0
0 1 0 0 | 0
0 1 0 1 | 1
0 1 1 0 | 0
0 1 1 1 | 0
1 0 0 0 | 1
1 0 0 1 | 0
1 0 1 0 | 0
1 0 1 1 | 0
1 1 0 0 | 0
1 1 0 1 | 0
1 